# Currents Ecosystem Model (cadCAD) | CURRENTS

### Model introduction


### Assumptions
 
- 

### Constraints / Scope


### Model extension



# 0. Dependencies

In [1]:
# Standard libraries: https://docs.python.org/3/library/
import math
from numpy import random
import numpy as np

# Analysis and plotting modules
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from random import normalvariate

# cadCAD configuration modules
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment

# cadCAD simulation engine modules
from cadCAD.engine import ExecutionContext, Executor

# Network modules
import networkx as nx


### Monte Carlo 

In [15]:
MONTE_CARLO_RUNS = 1

#flexible way to create unique seeds for each monte carlo run
seeds = [random.RandomState(i) for i in range(MONTE_CARLO_RUNS)] 

## Graph

We model a directed graph with agents as nodes. A network is a set of items (nodes or vertices) connected by edges or links. 
We represent a network by a graph (N, g), which consists of a set of nodes N = {1, . . . , n}.

#### Node types
* Agent

An agent is a user in the CURRENTS ecosystem. They can be one of many types, including clients, relays and hosts.

* Host
* Relay
* Client

#### Edge types
* Bandwidth Allocation
* Transactions
* Signal quality (a function of distance)

### Initialise Agents Network

In [16]:
# Generate an empty directional graph. Direction indicates transaction direction (e.g. Client -> Relay -> Host)
G = nx.MultiDiGraph()

# Size of population
POTENTIAL_USERS = 100

# Add agents to graph, with no edges. 
for agent in range(POTENTIAL_USERS):
    G.add_node(agent, type="potential_user")



# 1. State Variables & System Parameters

In [17]:
timeHorizon = 365 # (Days)

initial_state = {
    'clients': int(1),
    'hosts': int(1), 
    'avg_price': int(5), # (ZAR/Mbps/Day) The price hosts set for connectivity
    'potential_users': int(10000),
    
    'network_capacity': int(0), # (Mbps) Total host capacity
    'network_demand': int(0), # (Mbps) Estimated total demand for connectivity
    'network_allocation': int(0), # (Mbps) Actual allocated demand
    'network_penetration': int(0), # (%) Population servicable by hosts (a function of network coverage)
    

}


system_params = {
    
    'initial_population': [10000],
    'onboarding_coefficient': [0.1], # (%) adjust onboarding rate 
    
    'client_competitor_price': [0.5], # (ZAR/Mbps/Day) equivelent to AVG host service 
    'avg_client_allocation': [10, 50], # (Mbps) 'Bandwidth' 
    'client_registration_delay': [7], # (Day)
    
    'host_line_cost': [0.06], # (ZAR/Mbps/Day) ISP package
    'avg_host_line': [1000], # (Mbps) Backhaul capacity
    'network_inefficiencies': [0.25], # (%) losses due to hardware inefficiencies, downtime, environemnt, etc. 
    'host_setup_delay': [60], # (Day)
    'host_technical_difficulty': [0.25] # (%) threshold for host onboarding
    
        
}

# 2. Policy functions

In [18]:
# Policy functions return an Input/Signal (Python dictionary) which is used by State Update functions to update the state.

def p_client_adoption(params, substep, state_history, previous_state):
    
    return {'clients': clients}


def p_host_adoption(params, substep, state_history, previous_state):
    
    return {'hosts': hosts}


def p_demand(params, substep, state_history, previous_state):
    
    return {'demand': demand}


def p_price(params, substep, state_history, previous_state):
    
    return {'price': price}


def p_allocation(params, substep, state_history, previous_state):
    
    return {'allocation': allocation}


# 3. State Update Functions

In [19]:
# Comment
def s_clients(params, substep, state_history, previous_state, policy_input):
    clients = policy_input['clients'] 
    return ('clients', clients)

def s_hosts(params, substep, state_history, previous_state, policy_input):
    hosts = policy_input['hosts'] 
    return ('hosts', hosts)

def s_potential_users(params, substep, state_history, previous_state, policy_input):
    potential_users = policy_input['potential_users'] 
    return ('potential_users', potential_users)

def s_capacity(params, substep, state_history, previous_state, policy_input):
    capacity = policy_input['capacity'] 
    return ('capacity', capacity)

def s_demand(params, substep, state_history, previous_state, policy_input):
    demand = policy_input['demand'] 
    return ('demand', demand)

def s_allocation(params, substep, state_history, previous_state, policy_input):
    allocation = policy_input['allocation'] 
    return ('allocation', allocation)

def s_price(params, substep, state_history, previous_state, policy_input):
    price = policy_input['price'] 
    return ('price', price)

# 4. Partial State Update Blocks

In [20]:
partial_state_update_blocks = [
    {
        'policies': {
            'client_adoption': p_client_acquisition,
            'host_adoption': p_host_acquisition,
            'demand': p_demand,
            'price': p_price,
            'demand': p_demand,
            
        },
        'variables': {
            'clients': s_clients,
            'hosts': s_hosts,
            'potential_users': s_potential_users,
            'network_capacity': s_capacity,
            'network_demand': s_demand,
            'network_allocation': s_allocation,
            'avg_price': s_price
        }
    }
]

# 5. Simulation Execution

In [21]:
from cadCAD import configs
del configs[:] # Clear any prior configs


sim_config = config_sim({
    'N': 1,
    'T': range(timeHorizon),
    'M': system_params
})


experiment = Experiment()
experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)



exec_context = ExecutionContext()
run = Executor(exec_context=exec_context, configs=configs)

(system_events, tensor_field, sessions) = run.execute()


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (365, 10, 2, 8)
Execution Method: local_simulations
SimIDs   : [0, 0]
SubsetIDs: [0, 1]
Ns       : [0, 1]
ExpIDs   : [0, 0]
Execution Mode: parallelized


NameError: name 'clients' is not defined

# 6. Simulation Output Preparation

### Clean substeps

In [ ]:


# Convert system events dictionary into a pandas dataframe
# Get system events and attribute index
df = (pd.DataFrame(system_events)
      .assign(Days=lambda df: df.timestep)
      .query('timestep > 5')
     )

# Clean substeps
first_ind = (df.substep == 0) & (df.timestep == 0)
last_ind = df.substep == max(df.substep)
inds_to_drop = (first_ind | last_ind)
df = df.loc[inds_to_drop].drop(columns=['substep'])

# Attribute parameters to each row
df = df.assign(**configs[0].sim_config['M'])
for i, (_, n_df) in enumerate(df.groupby(['simulation', 'subset', 'run'])):
    df.loc[n_df.index] = n_df.assign(**configs[i].sim_config['M'])
    

### Check initial data

In [ ]:
simulation_result = pd.DataFrame(system_events)
simulation_result.tail(10)

### Convert data to JSON

In [ ]:
# See pandas reference for different formatting: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_json.html

sim_json = simulation_result.to_json(orient='records', lines=True)

sim_json

# 7. Plot data

### a) Plot name

<p> description </p>